<a href="https://colab.research.google.com/github/balloch/playingpretend/blob/colab_merge/Copy_of_Playing_Pretend_simpleai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making An Interactive TTRPG Story with simpleaichat

An update to ChatGPT on June 13th, 2023 allows the user to set a predefined schema to have ChatGPT output data according to that schema and/or take in an input schema and respond better to that data. This "function calling" as OpenAI calls it can be used as a form of tools, but the schema, enabled by a JSON-finetuning of ChatGPT, is much more useful for typical generative AI use cases, particularly when not using GPT-4.

OpenAI's [official demos](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb) for this feature are complicated, but with simpleaichat, it's very easy to support placing your own data

**NOTE: Ensuring input and output follows a complex predefined structure is very new in the field of prompt engineering and although it is very powerful, your mileage may vary.**


In [ ]:
!pip install -q simpleaichat

from simpleaichat import AIChat
import orjson
from rich.console import Console
from getpass import getpass

from typing import List, Literal, Optional, Union
from pydantic import BaseModel, Field

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.8.0 which is incompatible.


In [ ]:
api_key = ""

For the following cell, input your OpenAI API key when prompted. **It will not be saved to the notebook**.

In [ ]:
api_key = getpass("OpenAI Key: ")

In [ ]:
system_prompt = "You are a helpful assistant. You keep your answers brief. When asked for steps or a list you answer with an enumerated list. Only give the answer to the question, do not expound on your answers."

logic_system_prompt = """This is a logical, common sense, question answering task.
Your job is to answer questions as simply and correctly as possible.
When asked for steps or a list, please answer in an enumerated list.
Only give the answer to the question, do not expound on your answers."""

creative_system_prompt = """You are a children's story teller and game designer. Be creative but concise."""
better_system_prompt = """You are a children's story teller and game designer.

Write a setting description and two character sheets for the setting the user provides.

Rules you MUST follow:
- All names you create must be creative and unique. Always subvert expectations.
- Include as much information as possible in your response."""

original_system_prompt = """You are a world-renowned game master (GM) of tabletop role-playing games (RPGs).

Write a setting description and two character sheets for the setting the user provides.

Rules you MUST follow:
- Always write in the style of 80's fantasy novels.
- All names you create must be creative and unique. Always subvert expectations.
- Include as much information as possible in your response."""

In [ ]:
# model = text-davinci-003  ## Doesn't work
# params = {"temperature": 0.0, "max_tokens": 100}
qa_ai = AIChat(system=system_prompt, model='gpt-3.5-turbo-0613', save_messages=False, api_key=api_key, params = {"temperature": 0.0})
creative_ai = AIChat(system=creative_system_prompt, model="gpt-3.5-turbo-0613", save_messages=True, api_key=api_key, params = {"temperature": 0.1})

theme = 'Find Buried Pirate Treasure'

In [ ]:
# theme = 'slaying a dragon in a cave'

#### Creative Bot #####
# prompt = original_decomp.format(theme=theme)

# response = creative_ai(theme)
# print('response: ', response)


In [ ]:
# corrector_prompt = "Only give the answer to the question, do not expound on your answers"
# original_decomp = "What are the steps required to {theme}?"

# #### Logic Bot #####
# prompt = original_decomp.format(theme=theme)

# response = qa_ai(prompt)
# print('first response: ', response)
# response_list = response.split('\n')
# if len(response_list[0]) > 30:
#     correction = qa_ai(corrector_prompt)
#     ## Sometimes the correction contains the answer, sometimes it just apologizes
#     response_list = correction.split('\n')
#     if len(response_list) <= 2:
#         correction = qa_ai(prompt)
#         response_list = correction.split('\n')
#     print('correction: ', correction)
# else:
#     response_list = response.split('\n')

# response_list = [s.lstrip('0123456789 .').rstrip('.') for s in response_list]
# for list_item in response_list:
#     qa_ai(prompt)

In [ ]:
# AIChat(api_key=api_key)
# AIChat(system=system_prompt, model=model, console=True, save_messages=True, api_key=api_key)

In [ ]:
# response = ai("Python software development and beach volleyball")
# print(response)

Evocative, but a bit disorganized. If we instead allow for structured data output that follows specifications, then we'll have a lot more flexibility both in terms of directing generation, and playing with the resulting output.

That's where the `schema_output` field comes in when generating. If you construct a schema with pydantic ,which is also installed with simpleaichat as it is used heavily internally, then the output will generally follow the schema you provide!

We want an output containing the setting **name** and **description**, along with a list of player characters. Since each character has its own attributes, and we may want the model to generate multiple chatacters, we'll define a schema for that first.

We must also set a description for each field, can provide further hints to ChatGPT for how to guide generation. There is a _lot_ of flexibility here!


In [ ]:
class player_character(BaseModel):
    name: str = Field(description="Character name")
    race: str = Field(description="Character race")
    job: str = Field(description="Character class/job")
    story: str = Field(description="Three-sentence character history")
    feats: List[str] = Field(description="Character feats")
    equipment: List[str] = Field(description="Character equipment")

An important note: with this new ChatGPT model, the

1.   List item
2.   List item

fields are generated _in order_ at runtime according to the schema. Therefore, the order of the fields specified is important! Try to chain information!

Now we can build the schema for the TTRPG we will send to ChatGPT. In this case, we will order the fields such that we generate `description` and then `name`, as the former will be more imaginative and the latter can be infered from it. We will also add a list of player characters using the player character schema above.

Lastly, we will also include a docstring for the schema class; the specifics don't matter but it can provide another editorial hint.


In [ ]:
class write_ttrpg_setting(BaseModel):
    """Write a fun and innovative live-action role playing scenario"""

    description: str = Field(
        description="Detailed description of the setting in the voice of the game master"
    )
    quest: str = Field(description="The challenge that the players are trying to overcome.")
    success_metric: str = Field(description="A concise, one-sentence explanation of how the players will know they have succeeded at their quest.")
    init: str = Field(description="Where does the adventure begin?")  ####### This should be determined by the real world
    name: str = Field(description="Name of the setting")
    pcs: List[player_character] = Field(description="Player characters of the game")

In [ ]:
# response_structured = creative_ai(
#     theme, output_schema=write_ttrpg_setting
# )

# orjson.dumps preserves field order from the ChatGPT API
# print(orjson.dumps(response_structured, option=orjson.OPT_INDENT_2).decode())

Since the output is structured, we can parse it as we want.

For example, if we just want the setting name:


In [ ]:
# response_structured['name']

In [ ]:
real_object_list = [
    'refrigerator_1',
    'refrigerator_2',
    'kitchen_island_1',
    'electric_kettle_1',
    'electric_kettle_2',
    'coffee_machine_1',
    'coffee_machine_2',
    'microwave_1',
    'microwave_2',
    'coffee_can_1',
    'tea_can_1',
    'door_1',
    'garbage_can_1',
    'garbage_can_door_1',
    'sink_1',
    'fork_1',
    'spoon_1',
    'knife_1',
]

objects_stripped = [s.rstrip('01234567890_').replace('_',' ') for s in real_object_list]
object_types = set(objects_stripped)
print(object_types)

real_object_locations = {
    'refrigerator_1':2,
    'refrigerator_2':3,
    'kitchen_island_1':5,
    'electric_kettle_1':4,
    'electric_kettle_2':4,
    'coffee_machine_1':5,
    'coffee_machine_2':6,
    'microwave_1':7,
    'microwave_2':7,
    'coffee_can_1':4,
    'tea_can_1':4,
    'door_1':1,
    'garbage_can_1':5,
    'garbage_can_door_1':5,
    'sink_1':5,
    'fork_1':4,
    'spoon_1':4,
    'knife_1':4,
}

themes = [
    'Being a Waiter/Waitress',
    'Being a Zookeeper',
    'Being a Florist',
    'Being a Actor/Actress',
    'Being a Chef',
    'Being a Doctor',
    'Being a Teacher',
    'Being a Detective',
    'Running a Farm',
    'Running a zoo',
    'being in the circus',
    'being an astronaut and going to outer space',
    'Saving the day as a superhero',
    'Fairies in a fairie tale',
    'Make something as a construction worker'

]

{'refrigerator', 'microwave', 'knife', 'tea can', 'coffee can', 'electric kettle', 'garbage can door', 'fork', 'sink', 'door', 'kitchen island', 'coffee machine', 'garbage can', 'spoon'}


In [ ]:
def tidy_llm_list_string(llm_list_string, strip_nums=True):
    if type(llm_list_string) is str:
        llm_list = llm_list_string.split('\n')
        if len(llm_list) == 1 and len(llm_list_string.split(',')) > 1:
            print('WARNING: splitting on commas')  # TODO balloch: make this a real warning
            llm_list = llm_list_string.split(',')
    else:
        llm_list = llm_list_string
    if strip_nums:
        return [s.lstrip('0123456789 .').rstrip(' .').strip() for s in llm_list]
    return [s.strip() for s in llm_list]



# Ask a logic bot

In [ ]:
story = creative_ai(f"Write a short story featuring two friends, Astro and Playmate, about {theme} that a 5 year old would understand and enjoy")
print('story, ', story)
gen_loc = qa_ai(f"Given the story {story}, where does the story take place? If you can't tell from the story, just say 'Location Name: The story world' \n Example: 'Location Name: <example name>' ")
gen_loc = gen_loc.replace('Location Name:','').strip()
print('##gen_loc?, ', gen_loc)

# story_init_loc = qa_ai(f"Given the story '{story}' \n that takes place in {gen_loc}, what specific location within {gen_loc} where the characters most likely begin the story, before they {theme}. Only answer with the Location Name. \n Example: 'Location Name: <example_location_inside_{gen_loc}>'")
story_init_loc = creative_ai(f"Given the story which takes place in {gen_loc}, what specific location within {gen_loc} are the characters most likely begin the story, before they {theme}? Only answer with the Location Name. \n Example: 'Location Name: <example_location_inside_{gen_loc}>'")
# story_init_loc = story_init_loc.split('\n')
story_init_loc = story_init_loc.replace('Location Name:','').strip()
print('##story init loc, ', story_init_loc)
# story_init_loc[1] = story_init_loc[1].replace('Location Type: ','').strip()

# story = qa_ai(f"Rewrite the story {story}, using proper nouns instead of pronouns wherever possible.")

story_points = qa_ai(f"Given the story {story}, only using sentences with one clause list the five most important actions the characters made in the story to {theme}.") # Use proper nouns instead of pronouns wherever possible")
story_points_list = tidy_llm_list_string(story_points)
print('##story_points, ', story_points_list)
creative_story_points = creative_ai(f"Given the story {story}, only using sentences with one clause list the five most important actions the characters made in the story to {theme}.") # Use proper nouns instead of pronouns wherever possible")
creative_story_points_list = tidy_llm_list_string(creative_story_points)
print('##creative_story_points_list, ', creative_story_points_list)



story,  Once upon a time, in a small town by the sea, there were two best friends named Astro and Playmate. They loved going on adventures together and dreaming about finding buried pirate treasure.

One sunny day, Astro and Playmate decided to embark on their biggest adventure yet. They packed their backpacks with snacks, a treasure map, and a trusty shovel. With excitement in their hearts, they set off to find the hidden treasure.

Following the map's clues, they walked through a dense forest, crossed a sparkling river, and climbed a steep hill. Finally, they reached a mysterious cave. Inside, they found a secret passage that led them deep underground.

In the dimly lit tunnel, Astro and Playmate discovered glittering gems and ancient artifacts. But their eyes were set on the ultimate prize—the buried pirate treasure. They continued their journey, determined to find it.

After what felt like hours, they reached a large chamber filled with golden coins, sparkling jewels, and shiny swo

In [ ]:
import inspect
import pandas as pd


#TODO balloch: keeping a record of this with something like pandas will allow you to do lookups according to similar features
class Task: #(BaseModel)

    def __init__(self, name, expected_start_location=None, expected_visit_location=[], objects_required=[], primitive_fn=None, subtasks=[], effects=None, root=False):
        # super().__init__(**kwargs)
        self.name = name
        # self.terms = terms  ## preconditions
        self.expected_start_location = expected_start_location
        self.expected_visit_location = expected_visit_location
        self.objects_required = objects_required
        self.primitive_fn = primitive_fn
        self.subtasks = subtasks
        self.effects = effects
        self.root = root

    def __hash__(self):
        return hash(frozenset((self.name, self.primitive_fn)))

    def __eq__(self, other):
        return isinstance(other, Task) and self.name == other.name and self.primitive_fn == other.primitive_fn

    def __ne__(self, other):
        return not self.__eq__(other)


class Object: #(BaseModel)
    object_coll = {}
    def __init__(self, name, loc):
        self.name = name
        self.init_loc = loc
        iter = self._put(loc)
        self.init_iter = iter

    def _put(self, new_loc):
        if (self.name, new_loc) in self.object_coll:
            self.object_coll[(self.name,new_loc)].append(self)
        else:
            self.object_coll[(self.name,new_loc)]=[self]
        self.loc = new_loc
        len(self.object_coll[(self.name,new_loc)])
        self.iter = iter
        return iter

    def move(self, new_loc):
        self.object_coll[(self.name,self.loc)].remove[self.iter]
        self._put(new_loc)

    def __hash__(self):
        return hash(frozenset((self.name, self.init_loc, self.init_iter)))

    def __eq__(self, other):
        return isinstance(other, Object) and self.name == other.name and self.init_loc == other.loc and self.init_iter == other.init_iter

    def __ne__(self, other):
        return not self.__eq__(other)



# def add_object_to_df(df, class_obj):
#     ## check if the DF is associated with this type of class:
#     print(type(df['obj'].iloc[0]))
#     print(type(class_obj))
#     if isinstance(class_obj, df['obj'].iloc[0].__class__):
#         df = df.append([[class_obj] + list(class_obj.__dict__.values())])
#     return df



## Dummy robot API functions
## Assumption: manually need to specify expected effect for primitives
class RobotAPI:
    def __init__(self, init_loc, real_graph=None, inventory=None):
        self.curr_loc = init_loc
        self.graph = real_graph
        if inventory is None:
            self.inventory = {}  ## items are <hash: object>
        elif isinstance(inventory,dict):
            self.inventory = inventory
        else:
            raise TypeError('only "None" and "dict" allowed right now')
        self._errors = []

    def _move(self, dir='forward'):
        if dir != 'forward':
            raise NotImplementedError('Only forward motion currently implemented')
        return NotImplementedError('still using teleportation')

    def _turn(self, dir):
        if dir not in ('right', 'left'):
            raise NotImplementedError('Only left and right turns currently implemented')
        return NotImplementedError('still using teleportation')

    def go_To(self, loc):
        ## TODO balloch: currently teleport. Add A* and use _move and _turn
        if len(self._errors) > 0:
            self._errors = []
        self.curr_loc = loc
        print('Winning!')
        return self._errors

    def go_To_Object(self, obj, loc=None):
        obj_loc = obj.loc
        return self.go_To(obj_loc)

    def pick_Object(self, obj, id=None):
        if len(self._errors) > 0:
            self._errors = []
        if isinstance(obj, str) and (obj, self.curr_loc) in Object.object_coll:
            real_obj = Object.object_coll[obj, self.curr_loc][0]
        elif isinstance(obj, Object) and real_obj.loc == self.curr_loc:
            real_obj = obj
        else:
            self._errors.append(f'No object {real_obj} at {self.curr_loc}')
            return self._errors
        self.inventory[hash] = real_obj
        real_obj.move('inventory')
        return self._errors

    def put_Object(self, obj, id=None):
        if len(self._errors) > 0:
            self._errors = []
        ## Find object in inventory:
        if isinstance(obj, str):
            for hash, inv_obj in self.inventory.items():
                if inv_obj.name == obj:
                    real_obj = inv_obj
                    break
        elif isinstance(obj, Object):
            hash = obj.__hash__()
            if hash in self.inventory:
                real_obj = obj
        else:
            self._errors.append(f'No object {real_obj} in inventory')
            return self._errors
        del self.inventory[hash]
        real_obj.move(self.curr_loc)
        return self._errors

    def use_Object(self, obj, func=None, id=None, obj2=None, id2=None):
        if len(self._errors) > 0:
            self._errors = []
        if isinstance(obj, str):
            for hash, inv_obj in self.inventory.items():
                if inv_obj.name == obj:
                    real_obj = inv_obj
                    break
        elif isinstance(obj, Object):
            hash = obj.__hash__()
            if hash in self.inventory:
                real_obj = obj
        else:
            self._errors.append(f'No object {real_obj} in inventory')
            return self._errors
        if real_obj.use_fn is not None:
            real_obj.use_fn(func=func)
        else:
            self._errors.append(f'Object {real_obj} has no such use')
            return self._errors
        return self._errors



## Imaginary matching primitive_fns
## TODO balloch there should be some clever way to make this automatic with LLMs, shouldn't have to specify
class ImaginaryAgent:
    def __init__(self, init_loc, imaginary_graph=None, inventory=None):
        self.curr_loc = init_loc
        self.graph = imaginary_graph
        if inventory is None:
            self.inventory = {}  ## items are <hash: object>
        elif isinstance(inventory,dict):
            self.inventory = inventory
        else:
            raise TypeError('only "None" and "dict" allowed right now')
        self._errors = []

    def go_To(self, loc):
        ## TODO balloch: currently teleport. Add A* and use _move and _turn
        if len(self._errors) > 0:
            self._errors = []
        self.curr_loc = loc
        print('Winning!')
        return self._errors

    def go_To_Object(self, obj, loc=None):
        obj_loc = obj.loc
        return self.go_To(obj_loc)

    def pick_Object(self, obj, id=None):
        if len(self._errors) > 0:
            self._errors = []
        if isinstance(obj, str) and (obj, self.curr_loc) in Object.object_coll:
            real_obj = Object.object_coll[obj, self.curr_loc][0]
        elif isinstance(obj, Object) and real_obj.loc == self.curr_loc:
            real_obj = obj
        else:
            self._errors.append(f'No object {real_obj} at {self.curr_loc}')
            return self._errors
        self.inventory[hash] = real_obj
        real_obj.move('inventory')
        return self._errors

    def put_Object(self, obj, id=None):
        if len(self._errors) > 0:
            self._errors = []
        ## Find object in inventory:
        if isinstance(obj, str):
            for hash, inv_obj in self.inventory.items():
                if inv_obj.name == obj:
                    real_obj = inv_obj
                    break
        elif isinstance(obj, Object):
            hash = obj.__hash__()
            if hash in self.inventory:
                real_obj = obj
        else:
            self._errors.append(f'No object {real_obj} in inventory')
            return self._errors
        del self.inventory[hash]
        real_obj.move(self.curr_loc)
        return self._errors

    def use_Object(self, obj, func=None, id=None, obj2=None, id2=None):
        if len(self._errors) > 0:
            self._errors = []
        if isinstance(obj, str):
            for hash, inv_obj in self.inventory.items():
                if inv_obj.name == obj:
                    real_obj = inv_obj
                    break
        elif isinstance(obj, Object):
            hash = obj.__hash__()
            if hash in self.inventory:
                real_obj = obj
        else:
            self._errors.append(f'No object {real_obj} in inventory')
            return self._errors
        if real_obj.use_fn is not None:
            real_obj.use_fn(func=func)
        else:
            self._errors.append(f'Object {real_obj} has no such use')
            return self._errors
        return self._errors


    imag_fn_map = {
        'go_To': go_To,
        'go_To_Object': go_To_Object,
    }






# testing_df = pd.DataFrame([[story_tree] + list(story_tree.__dict__.values())], columns=['obj']+list(story_tree.__dict__.keys()))
# print(testing_df)
# print(add_object_to_df(testing_df, story_tree))



In [ ]:

init_robot_loc = 0
init_imaginary_loc = 'treehouse'
robot_api = RobotAPI(init_robot_loc)
im_agent = ImaginaryAgent(init_imaginary_loc)
## Start with a list of lists, then turn into a

real_tasks = []
imaginary_tasks = []

## Initialize API functions as primitive tasks
for primitive in inspect.getmembers(RobotAPI, predicate=inspect.isfunction):
    primitive_str = str(primitive[0])
    if primitive_str[0] != '_':
        ## Autoground:

        real = Task(
            name = primitive_str,
            primitive_fn = primitive[1],
            )
        # imaginary_fn = lambda
        imaginary = Task(
            name = primitive_str,
            primitive_fn = getattr(im_agent, primitive_str),
            )

        real_tasks.append(real)
        imaginary_tasks.append(imaginary)

print(real_tasks)
print(imaginary_tasks)

for obj, loc in real_object_locations.items():
    name = obj.rstrip('1234567890_').replace('_',' ')
    print(name)
    Object(name,loc)

print(Object.object_coll)
story_tree = Task(
    name=theme,
    expected_start_location=story_init_loc,
    expected_visit_location=[],
    objects_required= None,
    primitive_fn = None,
    subtasks=[],
    root=True,
)




[<__main__.Task object at 0x7cdc153f0040>, <__main__.Task object at 0x7cdc153f0430>, <__main__.Task object at 0x7cdc153f0250>, <__main__.Task object at 0x7cdc153f02b0>, <__main__.Task object at 0x7cdc153f01f0>]
[<__main__.Task object at 0x7cdc153f0100>, <__main__.Task object at 0x7cdc153f0190>, <__main__.Task object at 0x7cdc153f0280>, <__main__.Task object at 0x7cdc153f02e0>, <__main__.Task object at 0x7cdc153f0220>]
refrigerator
refrigerator
kitchen island
electric kettle
electric kettle
coffee machine
coffee machine
microwave
microwave
coffee can
tea can
door
garbage can
garbage can door
sink
fork
spoon
knife
{('refrigerator', 2): [<__main__.Object object at 0x7cdc153f1b40>], ('refrigerator', 3): [<__main__.Object object at 0x7cdc153f1e70>], ('kitchen island', 5): [<__main__.Object object at 0x7cdc153f0910>], ('electric kettle', 4): [<__main__.Object object at 0x7cdc15401090>, <__main__.Object object at 0x7cdc153f0940>], ('coffee machine', 5): [<__main__.Object object at 0x7cdc153f0

In [ ]:
## TODO balloch: automate this above
characters = ['Astro', 'Playmate']
lc = len(characters)
if lc > 1:
    the_characters = 'and ' + characters[-1]
    if lc > 2:
        the_characters = ', ' + the_characters
    for cha in characters[:lc-1]:
        the_characters += cha

story_point_list_present = [qa_ai(f"Convert the following sentence to present tense: {point}") for point in enumerate(story_points_list)]
print('##story_point_list_present, ', story_point_list_present)

state = {'time':0, 'loc':story_init_loc,} # 'loc_type':story_init_loc[1],}
print('##state, ', state)


# del creative_ai.default_session.messages[3:] ## Only for colab debugging

# story_tree = {
#     'task_description': theme,
#     'expected_start_location' : story_init_loc,
#     'expected_visit_location' : [],
#     'objects_required': None,
#     'primitive_fn' : None,
#     'subtasks' : [],
#     'root' : True,
# }
# story_tree = Task(
#     'name'=theme,
#     'expected_start_location'=story_init_loc,
#     'expected_visit_location'=[],
#     'objects_required'= None,
#     'primitive_fn' = None,
#     'subtasks' = [],
#     'root' = True,
# )

init_loc = story_init_loc

for idx, point in enumerate(story_point_list_present):
    current_task = Task(
        name= point,
        expected_start_location= init_loc,
    )
    # Check primitives i.e. if they have to possess something to do this story point
    #############
    ##### Primitive Decomps:
    #############
    print('##point, ', point)

    ### Locations
    move_tf = qa_ai(f"True or False: it is possible the {the_characters} can successfully '{point}' while staying at {state['loc']}.") # \n context story: \n {story} \n .")
    print('##Unecessary to Move?, ', move_tf)
    if move_tf[0:4] == 'True':
        move_tf = True
    elif move_tf[0:5] == 'False':
        move_tf = False
    else:
        raise TypeError

    if move_tf is False:
        ## TODO balloch: Add a location precondition and a move subtask
        ## TODO balloch: the below should be a function than any AI can use, where the parameters passed are (1) prompt, 'while' criteria function and variables, attempt limit, and 'validation function'
        new_locs_list = []
        attempt = 0
        creative_ai.default_session.messages.append('temp')
        while not len(new_locs_list):
            del creative_ai.default_session.messages[-1]
            print(attempt)
            new_loc = qa_ai(f"In the context of the story {story}, what are the places within {gen_loc} where the characters would most likely need to travel from {state['loc']} to {point}? Only respond with the list of location names, nothing more. ")  #[Example]'Location Name: <example_location>'") ### TODO balloch: may need creative ai
            creative_loc = creative_ai(f"Given the story, what are the places within {gen_loc} where the characters would most likely need to travel from {state['loc']} to {point}? Only respond with the list of location names, nothing more")
            print('##next_loc?, ', new_loc)
            print('##creative_loc?, ', creative_loc)
            new_loc = new_loc.replace('Location Name:','').strip()
            new_locs_list = tidy_llm_list_string(new_loc, strip_nums=True)
            # new_locs_list = [s.strip() for s in new_locs_list]
            ## Exposition Check and remove
            # if len(new_locs_list) > 1:
            #     orig_len = len(new_locs_list)
            #     for i in reversed(range(orig_len)):
            #         if len(new_locs_list[i]) == 0 or new_locs_list[i][0] not in ('0','1','2','3','4','5','6','7','8','9'):
            #             print('deleting: ',new_locs_list[i])
            #             del new_locs_list[i]
            attempt += 1
            if attempt > 5:
                raise IndexError
        new_locs_list = tidy_llm_list_string(new_locs_list)  #[s.lstrip('0123456789 .').rstrip(' .') for s in new_locs_list]
        ## TODO balloch: ground at least one of these locations
        current_task.expected_visit_location = new_locs_list
        print('##new_locs_list, ', new_locs_list)
    ## Need to update the location per question
    if idx+1 < len(story_point_list_present):
        creative_future_loc = creative_ai(f"Given the story, what is the place within {gen_loc} that {the_characters} must be before they start to {story_point_list_present[idx+1]}? Only respond with the name of one location, nothing more")
        print('creative_future_loc, ', creative_future_loc)   # where is the most likely/best starting point for a task like ________

        init_loc = tidy_llm_list_string(creative_future_loc)[0]  #[s.lstrip('0123456789 .').rstrip(' .') for s in creative_future_loc.split('\n')][0]
        if False:  ## TODO: potentially better solution for the future
            init_loc = qa_ai(f"Choose the location category that is most likely best to start to {story_point_list_present[idx+1]}: [Categories] \n {creative_future_loc} \n Only respond with the location name, nothing more. ")  #[Example]'Location Name: <example_location>'") ### TODO balloch: may need creative ai
        init_loc = init_loc.replace('Location Name:','').strip()
        print('future_loc, ', init_loc)   # where is the most likely/best starting point for a task like ________

        # del creative_ai.default_session.messages[-1]

    ### Objects
    get_objects = qa_ai(f"List the physical objects referenced in the sentence. \n [Example] \n Sentence: 'Do research and find evidence or maps that lead to the treasures whereabouts' \n Objects: 1. evidence \n 2. map \n 2. treasure. \n [Query] \n Sentence: '{point}' \n Objects: ")
    get_objects = tidy_llm_list_string(get_objects)  ## TODO balloch make this a lambda
    print('##get_objects, ', get_objects)
    # need_item_tf = qa_ai(f"True or False: In the story: \n {story} \n is it necessary to 'have' {get_objects} to {point_present}?")
    # need_item_tf = qa_ai(f"In the story: \n {story} \n For the characters to {point_present} what physical items are necessary besides {get_objects}?")  ## Doesn't work tooo complex
    current_task.objects_required = get_objects
    obj_uses = []
    if len(get_objects) > 0:
        for obj in get_objects:
            obj_possess_tf = qa_ai(f"True or False: In the sentence '{point}', the {obj} is possessed or acquired. ")
            print('###', obj, "possessed or acquired: ", obj_possess_tf)
            obj_use_tf = qa_ai(f"What one word best describes how the {the_characters} use {obj} in {point}'? ")
            print('###', obj, "how used: ", obj_use_tf)

        ## TODO balloch: try ground objects. if no real objects remaining, announce imaginary
        ## TODO balloch: add a go_to_object +  pick_object subtask

    ## deepen:


    ## Need to update the objects per question
    # if idx+1 < len(story_point_list_present):
    #     future_precon = qa_ai(f"In the context of the story {story}, what objects must {the_characters} interact with before they can reasonably {story_point_list_present[idx+1]}? Only respond with the object names, nothing more. ")  #[Example]'Location Name: <example_location>'") ### TODO balloch: may need creative ai
    state['loc'] = init_loc
    state['time'] += 1
    print('##state,' , state)

    story_tree.subtasks.append(current_task)

    # print(f'point_{idx}: ', subdecomp)
    # subdecomp = qa_ai(f"Given the story {story}, in as few steps as possible explain to a 5 year old how to {point_present}")
    # print(f'point_{idx}: ', subdecomp)


##story_point_list_present,  ['They pack their backpacks with snacks, a treasure map, and a trusty shovel.', "Follow the map's clues through a dense forest, cross a sparkling river, and climb a steep hill.", 'They discover a mysterious cave and find a secret passage that leads them deep underground.', 'Explores the dimly lit tunnel and discovers glittering gems and ancient artifacts.', 'They reach a large chamber filled with golden coins, sparkling jewels, and shiny swords, where they find the buried pirate treasure.']
##state,  {'time': 0, 'loc': 'The Beach'}
##point,  They pack their backpacks with snacks, a treasure map, and a trusty shovel.
##Unecessary to Move?,  False.
0
##next_loc?,  1. The Beach
2. Their homes
##creative_loc?,  The House of Astro, The House of Playmate
##new_locs_list,  ['The Beach', 'Their homes']
creative_future_loc,  The Town Square
future_loc,  The Town Square
##get_objects,  ['snacks', 'treasure map', 'trusty shovel']
### snacks possessed or acquired:  Tru

In [ ]:
## Visualize
from graphviz import Digraph
from IPython.display import Image


def visualize_task_tree(root_task):
    dot = Digraph(comment="Task Tree")
    stack = [(root_task, None)]  # Use a stack for depth-first traversal

    while stack:
        task, parent_id = stack.pop()
        task_id = id(task)

        if parent_id is not None:
            dot.edge(str(parent_id), str(task_id))

        dot.node(str(task_id), task.name)

        for subtask in task.subtasks:
            stack.append((subtask, task_id))

    return dot


## Main visualization function
# Create your Task objects and define their relationships
root_task = Task("Root Task", subtasks=[
    Task("Subtask 1"),
    Task("Subtask 2", subtasks=[
        Task("Subtask 2.1"),
        Task("Subtask 2.2"),
    ]),
], root=True)

# Visualize the task tree and render the graph
dot_graph = visualize_task_tree(story_tree) #root_task)

# Render the graph as an image in the notebook
dot_graph.format = 'png'
graph_image = Image(dot_graph.render(filename='task_tree', format='png', cleanup=True))
graph_image


# PDF for noninteractive
# dot_graph.render("task_tree", view=True)  # Saves the graph as "task_tree.pdf" and opens it


In [ ]:
len(creative_ai.default_session.messages) ##.__dict__.keys())  ## UUID('89bc0a29-1be6-47b2-9e9c-78b07ac2f33b')


# creative_ai.default_session.messages.append('temp')

In [ ]:
corrector_prompt = "Only give the answer to the question, do not expound on your answers"
decomp = "In stories, what specific steps do characters take to do the task: '{task}'? \n Example: Task: Find pirate treasure \n Steps: \n  1. Research and gather information. \n 2. Follow clues or maps. \n 3. Overcome obstacles. \n 4. Dig or search. \n 5. Celebrate and enjoy the discovery."
get_objects = "What are the objects in the sentence '{task}'?"
get_actions = "What are the actions in the sentence '{task}'?"
action_afford = {
    'pick up':'pickupable',
    'put':'putdownable',
    }
common_sense_check = 'Is it unusual to {task}?'
colocation_execution = 'do {objects} need to be colocated to {task}?'


def ensure_succinct(prompt):
    correction = qa_ai(corrector_prompt)
    ## Sometimes the correction contains the answer, sometimes it just apologizes
    if len(correction.split('\n')) <= 2:
        correction = qa_ai(decomp.format(task=prompt))
    print('correction: ', correction)
    return correction



#### Logic Bot #####
# prompt = decomp.format(subtask=theme)

response = qa_ai(decomp.format(task=theme))
print('first response: ', response)
response_list = response.split('\n')
if len(response_list[0]) > 30:
    ensure_succinct(decomp.format(task=theme))
else:
    response_list = response.split('\n')
response_list = [s.lstrip('0123456789 .').rstrip('.') for s in response_list]  ## TODO balloch make this a lambda

for list_item in response_list:
    ## Get objects
    objects = qa_ai(get_objects.format(task=list_item) + corrector_prompt).split('\n')
    objects = [s.lstrip('0123456789 .').rstrip('.') for s in objects]
    # if len(objects) > 5:
    #     ensure_succinct(decomp.format(task=theme))
    # else:
    #     response_list = response.split('\n')

    print('objects, ', objects)
    ## Get actions
    actions = qa_ai(get_actions.format(task=list_item) + corrector_prompt).split('\n')
    actions = [s.lstrip('0123456789 .').rstrip('.') for s in actions]
    print('actions, ', actions)



In [ ]:
system_prompt_plan_assist = """You are assisting with planning a sequence of actions to an eventual goal state.

Rules you MUST follow:
- Always be as concise as possible.
- When told to choose from multiple choices, you will not answer with anything other than those choices.

If the following is a yes-no question, respond with yes or no, otherwise respond with a step-by-step process."""


In [ ]:
from simpleaichat.utils import fd


class Postconditions(BaseModel):
    character_name: str = fd("Character name")
    dialogue: str = fd("Dialogue from the character")


class Preconditions(BaseModel):
    description: str = fd(
        "Detailed setting or event description, e.g. The sun was bright."
    )

class Quest(BaseModel):
    description: str = fd(
        "The challenge that the players are trying to overcome."
    )



# class Event(BaseModel):
#     type: Literal["setting", "conversation"] = fd(
#         "Whether the event is a scene setting or a conversation by an NPC"
#     )
#     data: Union[Action, Precondition, Postcondition] = fd("Event data")
class Event(BaseModel):
    type: Literal["primitive", "module"] = fd(
        "Whether the event is a scene setting or a conversation by an NPC"
    )
    data: Union[Action, Precondition, Postcondition] = fd("Event data")


class write_ttrpg_story(BaseModel):
    """Write an award-winning TTRPG story"""

    events: List[Event] = fd("All events in a TTRPG campaign.")



In [ ]:
#Test interactive querying of path
ai_plan_assist = AIChat(system=system_prompt_plan_assist, model=model, save_messages=False, api_key=api_key)
input_ttrpg = write_ttrpg_setting.model_validate(response_structured)
setting = ai_plan_assist(input_ttrpg, input_schema=write_ttrpg_setting)
print(orjson.dumps(setting, option=orjson.OPT_INDENT_2).decode())
new_ask = input()

for i in range(10):
    response = ai_plan_assist(new_ask)
    print(orjson.dumps(response, option=orjson.OPT_INDENT_2).decode())
    new_ask = input()

# TODO: need to be able to parse the quest output and the init output
# TODO: add a constraint that says "We know we cannot act if we are not colocated"
# TODO: parse decomp, add them to action buffer, and provide them as a bank
# TODO: add an if statement that checks whether an *new* action is basically equivalent to existing actions
# TODO (reach goal): have async calls that breakdown forward-and-backward for bidirectional
# TODO: questions for decomposition need to be caviated so that there is something like "Do nothing"
# TODO: filter out suggestions that are (1) too vague or (2) don't move the story forward (maybe ask "is this something people care about in DnD?" ----*** checked, this works!)
# TODO: make the "list ask" an input option so that you do have to always do it
# NOTE: doesn't like to take input options as a possibily well.

Now add the HTN

# Logic reminders
We have a set of V of variable symbols, a set C of constant symbols and sets Fn of n-ary function symbols, also called operator symbols (*,+,-, etc).
The set of __terms__ is the set of V, C, and all possible combinations with function symbols F_N(t1,...,tn)
In mathematical logic, a __term__ denotes a mathematical object while a __formula__ denotes a mathematical fact
For example, ( x + 1 ) * ( x + 1 ) is a __term__ built from the constant 1, the variable x, and the binary function symbols + and *;
it is part of the atomic __formula__ ( x + 1 ) * ( x + 1 ) \geq 0 , which is considered atomic because there are no deeper logics inside it (like predicates).
A predicate is a relation: it is a formula with an associated symbol that can be evaluated for truth. in first order logic it may only contain terms.

# HTN overview
ordered task decomposition planner is an HTN planner that plans for tasks in the same order that they will be executed
### Components:
 a Task is a list of the form of [name, t_1, ...t_n], comprised of the task's name, and [t_1,...,t_n] are terms. The task is primitive if name is a primitive task symbol (a symbol whose first character is an exclamation point)

begins with:
   - an initial state-of-the-world and
   - the objective of creating a plan to perform a set of tasks (abstract representations of things that need to be done).

the planner recursively decomposes tasks into subtasks, stopping when it reaches **primitive tasks** that can be performed directly by agents.
In order to tell the planner how to decompose nonprimitive tasks into subtasks, it needs to have a set of **methods**,
  - each method is a schema for decomposing a particular kind (???) of task into a set of subtasks
      - provided that some set of preconditions is satisfied.
For each task, there may be more than one applicable method, and thus more than one way to decompose the task into subtasks.

Tasks are "executed" either directly by an __operator__ if they are primitive or by decomposition by a __method__ if they are non-primitive
A __method__ s a triple m = (NT, DEC, P), where NT is a non-primitive task, DEC is a totally-ordered list of tasks called a decomposition of NT,
and P (the set of preconditions) is a boolean formula of first-order predicate calculus.
An __operator__ is a triple o=(PT,DEL,ADD), where PT is a primitive task,
and DEL and ADD are the sets of logical atoms that will be respectively deleted from and added to the world state when the operator is executed.
All variables in DEL and ADD must appear in the argument list of PT.

**The magic sauce: how do we know which task to associate with which method?**



Query LLM: "How does the child win the game/succeed in the adventure. I.e. What is the core challenge, and how specifically will we know that the challenge has been overcome"

### Define important concepts
 for all game concepts:
   Define detailed concept (game, quest character, etc)
   What are the key elements of that concept ()
         This should kinda be defined by the needs of the planner right?

### Ground important concepts
for each detailed concept:


## Backward chain preconditions of the game
Init: Game objective task
Init: Ask LLM for the steps necessary for accomplishing the objective task. This + primitives forms the initial set of tasks

For all no primitive tasks in Buffer, DFS:
   - Define next subtask that satisfies task:
       - If tasks exist in prior task list memoize,
       - Else query LLM (with masked sequence?) and add task to list
   - If defined task is primitive, assign and iterate out
   - Else deepen
Return solution tree


### In the case of change to the domain/violated assumptions
Init: Matrix of all preconditions, and a function that points to the node of the solution tree  based on the inputs to that pointer
_[NOTE: I guess this would kinda be a hash map? the key is a hash of the preconditions, and the value is the pointer to the tree node (which as data has a pointer to the task)]_


In [ ]:
primitive_actions = ['move_forward', 'turn45','goto', 'lookat']

In [ ]:
def find_matching_primitive():
    raise NotImplementedError

def decomp_with_ai():
    raise NotImplementedError


task_set = {}

task_sequence = []
goal_postcondition = response_structured["success_metric"]
goal_reachable = False

next_postcondition = goal_postcondition
first_precondition = response_structured["init"]

while not goal_reachable:
    primitive_match = find_matching_primitive()
    if primitive_match is None:
        decomp_with_ai()
    else:
        task_sequence.append(primitive_match)
        next_postcondition = primitive_match.precondition

    if first_precondition == task_sequence[0]:
        goal_reachable = True




## Structured Output and Structured Input

Now that we have a schema for a TTRPG setting, we can use the same hints we defined to help generation of a TTRPG adventure!

First, we convert the structured `dict` data to a pydantic object with that schema with `parse_obj`:


In [ ]:
input_ttrpg = write_ttrpg_setting.model_validate(response_structured)

Next, we define a schema for a list of events. To keep things simple, we'll just do **dialogue** and **setting** events. (a proper TTRPG would likely have a more detailed combat system!)

There are a few other helpful object types you can use to control output:

- `Literal`, to force a certain range of values.
- `Union` can be used to have the model select from a set of schema. For example we have one schema for `Dialogue` and one schema for `Setting`: if unioned, the model will use only one of them, which allows for token-saving output.

Lastly, if the `Field(description=...)` pattern is too wordy, you can use `fd` which is a shortcut.


In [ ]:
from simpleaichat.utils import fd


class Dialogue(BaseModel):
    character_name: str = fd("Character name")
    dialogue: str = fd("Dialogue from the character")


class Setting(BaseModel):
    description: str = fd(
        "Detailed setting or event description, e.g. The sun was bright."
    )

class Quest(BaseModel):
    description: str = fd(
        "The challenge that the players are trying to overcome."
    )



class Event(BaseModel):
    type: Literal["setting", "conversation"] = fd(
        "Whether the event is a scene setting or a conversation by an NPC"
    )
    data: Union[Dialogue, Setting] = fd("Event data")


class write_ttrpg_story(BaseModel):
    """Write an award-winning TTRPG story"""

    events: List[Event] = fd("All events in a TTRPG campaign.")

Lastly, we'll need a new system prompt since we have a different goal.


For the final call, we will need the parsed `input_ttrpg` object as the new "prompt", plus the `write_ttrpg_setting` schema used to build it as the `input_schema`.

Putting it all together:


In [ ]:
ai_plan_assist = AIChat(system=system_prompt_plan_assist, model=model, save_messages=False, api_key=api_key)

response_story = ai_plan_assist(
    input_ttrpg, input_schema=write_ttrpg_setting, output_schema=write_ttrpg_story
)

print(orjson.dumps(response_story, option=orjson.OPT_INDENT_2).decode())

{
  "events": [
    {
      "type": "setting",
      "data": {
        "description": "In the land of Pythoria, a mystical realm where the power of code reigns supreme, a group of software developers find themselves transported to a parallel world where their coding skills are put to the ultimate test. In this world, the lines of code they write have the power to shape reality itself. But they are not alone in this journey, as they are joined by a team of beach volleyball players who have been granted extraordinary powers by the ocean gods. Together, they must navigate the treacherous landscapes of Pythoria, battle fearsome creatures spawned from debugging nightmares, and uncover the secrets of this realm. Will they triumph and find their way back home, or will they be forever lost in the world of Python software development and beach volleyball?"
      }
    },
    {
      "type": "conversation",
      "data": {
        "character_name": "Aurora",
        "dialogue": "We find ourselve

Now that we have a structured output, we can output it like a story, with custom and consistent formatting!

In [ ]:
c = Console(width=60, highlight=False)

for event in response_story["events"]:
    data = event["data"]
    if event["type"] == "setting":
        c.print(data["description"], style="italic")
    if event["type"] == "conversation":
        c.print(f"[b]{data['character_name']}[/b]: {data['dialogue']}")

In the land of Pythoria, a mystical realm where the power of
code reigns supreme, a group of software developers find 
themselves transported to a parallel world where their 
coding skills are put to the ultimate test. In this world, 
the lines of code they write have the power to shape reality
itself. But they are not alone in this journey, as they are 
joined by a team of beach volleyball players who have been 
granted extraordinary powers by the ocean gods. Together, 
they must navigate the treacherous landscapes of Pythoria, 
battle fearsome creatures spawned from debugging nightmares,
and uncover the secrets of this realm. Will they triumph and
find their way back home, or will they be forever lost in 
the world of Python software development and beach 
volleyball?

Aurora: We find ourselves in a strange land, my friends. Our
coding skills are now our most powerful weapons. But fear 
not, for we are not alone. We are joined by a team of beach 
volleyball players who have also been granted extraordinary 
powers. Together, we shall face the challenges that lie 
ahead and find our way back home.

Kai: Indeed, Aurora. Our skills in coding and volleyball may
seem unconventional, but they make us a formidable team. 
With my enchanted arrows and tactical analysis, and your 
coding magic, we shall overcome any obstacle.

Aurora: But first, we must gather the rest of our party. Let
us seek out the others who have been transported to this 
realm and join forces.

Kai: Agreed, Aurora. Our first ally should be someone with 
expertise in debugging and problem-solving. They will be 
invaluable in this world where code shapes reality.

Aurora: I have heard of a legendary software engineer named 
Maelis who possesses such skills. Let us seek them out and 
convince them to join our cause.

Kai: Very well, Aurora. Let us embark on this quest and find
Maelis. They may hold the key to our success in this strange
realm.

Aurora: Our journey begins. May the power of code and sands 
guide us.

Kai: And may our skills in coding and volleyball bring us 
victory.

Aurora: Onward, my friends. To find Maelis and unravel the 
mysteries of Pythoria!

In [ ]:
## The Real World
## I imagine this as a graph of objects with properties, but we can start with a set
SceneGraph = {
    refrigerator_1,
    refrigerator_2,
    refrigerator_3,
    kitchen_island_1,
    electric_kettle_1,
    electric_kettle_2,
    coffee_machine_1,
    coffee_machine_2,
    microwave_1,
    microwave_2,
    folgers_coffee_can_1,
    lipton_tea_can_1,
    door_south_1,
    door_north_1,
    garbage_can_1,
    garbage_can_door_1,
    sink_1,
}



## Embodied Interactive Story Telling
## Starting information

players = ["Human","Astro"] ## NOT IMPLEMENTED YET
character_names = ["Agent Cool Kid", "AstroWatson"]
storyteller_name = "Astro"
quest = "Find and defuse the stink bomb before the clock gets to zero, and figure out who planted it."
word_limit = 50  # word limit for task brainstorming
lesson = "Notice when others need help and help them" ## NOT IMPLEMENTED YET

game_description = f"""Here is the topic for an imaginatery interactive game: {quest}.
        The characters are: {*character_names,}.
        The story is directed and managed by the storyteller, {storyteller_name}."""


In [ ]:
# model = "gpt-3.5-turbo-0613"
# ai = AIChat(system=system_prompt, model=model, save_messages=False, api_key=api_key)

template = """You are a senior staff role playing game designer.
You use a tone that is technical, child friendly, and creative.
You are direct and brief in your answers,
but more detailed when you need to be

{history}
Human: {human_input}
Assistant:"""

system_prompt_event = """You are a world-renowned game master (GM) of
family-friendly role-playing games (RPGs).

Write a complete three-act story in at least 9 events
with a shocking twist ending using the data from the
input_ttrpg function.

Write the player characters as a TTRPG party
fighting against a new evil.

In the second (2nd) event, the party must be formed.

Rules you MUST follow:
- Always write in the style of 80's fantasy novels.
- All names you create must be creative and unique. Always subvert expectations."""

ai_2 = AIChat(system=system_prompt_event,
              model=model,
              save_messages=False,
              api_key=api_key)

input_ttrpg = write_ttrpg_setting.model_validate(response_structured)

response_story = ai_2(
    input_ttrpg,
    input_schema=write_ttrpg_setting,
    output_schema=write_ttrpg_story
)



In [ ]:
# response = ai("Find and defuse the stink bomb before the clock gets to zero, and figure out who planted it.",
#             #   input_schema=write_ttrpg_setting,
#               output_schema=write_ttrpg_setting
            #   )
print(orjson.dumps(response_story, option=orjson.OPT_INDENT_2).decode())

{
  "events": [
    {
      "type": "setting",
      "data": {
        "description": "Welcome to the world of Pytharia, a land where the art of software development and the thrill of beach volleyball collide. In this unique setting, players will take on the roles of mighty programmers and skilled volleyball players, using their wit, agility, and teamwork to conquer challenges both on and off the court. Pytharia is a realm where coding prowess and athletic ability are equally revered, and only those who excel in both domains can truly claim the title of champion. Prepare to embark on a quest filled with epic coding battles, intense volleyball matches, and unexpected twists and turns. The fate of Pytharia rests in your hands. Will you rise to the challenge and become a legend?"
      }
    },
    {
      "type": "conversation",
      "data": {
        "character_name": "Aurelia Swiftstrike",
        "dialogue": "Xander, we must assemble a team of programmers and volleyball players to co

In [ ]:
# print(response.__dict__.keys())
print(response["names"])

TypeError: ignored

## MIT License

Copyright (c) 2023 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.